<a href="https://colab.research.google.com/github/psyuktha/Reccomendation_system/blob/main/Movie_reccomendation_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
!kaggle datasets download -d itschannel/telugu-movies-dataset

Dataset URL: https://www.kaggle.com/datasets/itschannel/telugu-movies-dataset
License(s): unknown
  0% 0.00/106k [00:00<?, ?B/s]
100% 106k/106k [00:00<00:00, 47.8MB/s]


In [3]:
!unzip telugu-movies-dataset.zip

Archive:  telugu-movies-dataset.zip
  inflating: TeluguMovies_dataset.csv  


In [4]:
df2=pd.read_csv('/content/TeluguMovies_dataset.csv')
df2.head()

,Unnamed: 0,Movie,Year,Certificate,Genre,Overview,Runtime,Rating,No.of.Ratings
0,0,Bahubali: The Beginning,2015.0,UA,"Action, Drama","In ancient India, an adventurous and darin...",159,8.1,99114
1,1,Baahubali 2: The Conclusion,2017.0,UA,"Action, Drama","When Shiva, the son of Bahubali, learns ab...",167,8.2,71458
2,2,1 - Nenokkadine,2014.0,UA,"Action, Thriller",A rock star must overcome his psychologica...,170,8.1,42372
3,3,Dhoom:3,2013.0,UA,"Action, Thriller","When Sahir, a circus entertainer trained i...",172,5.4,42112
4,4,Ra.One,2011.0,U,"Action, Adventure, Sci-Fi",When the titular antagonist of an action g...,156,4.6,37211


In [5]:
import pandas as pd
df=pd.read_csv('/content/TeluguMovies_dataset.csv',usecols=['Movie','Genre','Rating'])
df.head()

,Movie,Genre,Rating
0,Bahubali: The Beginning,"Action, Drama",8.1
1,Baahubali 2: The Conclusion,"Action, Drama",8.2
2,1 - Nenokkadine,"Action, Thriller",8.1
3,Dhoom:3,"Action, Thriller",5.4
4,Ra.One,"Action, Adventure, Sci-Fi",4.6


In [6]:

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

# Check if 'Genre' column contains any float values
if df['Genre'].apply(type).eq(float).any():
    print("Warning: 'Genre' column contains float values. Attempting to convert them to strings.")
    df['Genre'] = df['Genre'].astype(str)

# Transform the 'Genre' column
genre_vectors = mlb.fit_transform(df['Genre'].str.split('|'))

# Create a new DataFrame from the genre vectors
df_genre = pd.DataFrame(genre_vectors, columns=mlb.classes_)

# Concatenate the original DataFrame with the new genre DataFrame
df = pd.concat([df, df_genre], axis=1)

df.head()

,Movie,Genre,Rating,Action,"Action, Adventure","Action, Adventure, Comedy","Action, Adventure, Drama","Action, Adventure, Family","Action, Adventure, Fantasy","Action, Adventure, Romance",...,Mystery,"Mystery, Romance, Thriller","Mystery, Thriller",Romance,"Romance, Comedy, Drama","Romance, Drama","Romance, Thriller",Sport,Thriller,nan
0,Bahubali: The Beginning,"Action, Drama",8.1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Baahubali 2: The Conclusion,"Action, Drama",8.2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1 - Nenokkadine,"Action, Thriller",8.1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Dhoom:3,"Action, Thriller",5.4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ra.One,"Action, Adventure, Sci-Fi",4.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
from sklearn.decomposition import TruncatedSVD
movie_features_df_matrix = csr_matrix(df_genre.values)
from sklearn.neighbors import NearestNeighbors

svd = TruncatedSVD(n_components=171)
svd_matrix = svd.fit_transform(movie_features_df_matrix)
corr = np.corrcoef(svd_matrix)
corr.shape



(1400, 1400)

In [60]:
# Transform the csr_matrix to reduced dimensions
transformed_matrix = svd.transform(movie_features_df_matrix)


In [78]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

def recommend_movies_based_on_genre(movie_index, transformed_matrix, n_recommendations=5):
    # Calculate cosine similarity between the target movie and all other movies
    similarity = cosine_similarity([transformed_matrix[movie_index]], transformed_matrix)[0]

    # Get indices of movies sorted by similarity (excluding the target movie itself)
    similar_movies = np.argsort(similarity)[::-1]
    similar_movies = [movie for movie in similar_movies if movie != movie_index]

    # Return top-N similar movies
    return similar_movies[:n_recommendations]

# Example: Get recommendations for a random movie
random_movie_index = np.random.randint(0, transformed_matrix.shape[0])
print(random_movie_index)
recommendations = recommend_movies_based_on_genre(random_movie_index, transformed_matrix)
print(f"Top recommendations for movie index {df.iloc[random_movie_index][0:2]}")
for i,j in enumerate(recommendations):
   print(i+1,":",df.iloc[j][0:2])


31
Top recommendations for movie index Movie                        Mahanati
Genre    Biography, Drama            
Name: 31, dtype: object
1 : Movie              NTR: Kathanayakudu
Genre    Biography, Drama            
Name: 200, dtype: object
2 : Movie                NTR Mahanayakudu
Genre    Biography, Drama            
Name: 316, dtype: object
3 : Movie                    I Love You
Genre    Drama, Romance            
Name: 908, dtype: object
4 : Movie            7/G Rainbow Colony
Genre    Drama, Romance            
Name: 175, dtype: object
5 : Movie                         Bobby
Genre    Drama, Romance            
Name: 430, dtype: object
